In [211]:
import numpy as np
from matplotlib import pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
import scipy.linalg as la
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model  import LinearRegression
from sklearn.linear_model  import Lasso

In [212]:
file = open('A48168_Q001_data.p','rb')
D = pickle.load(file)
file.close()

In [213]:
x = D['X']
y = D['y']

In [214]:
print(x.shape)
print(y.shape)

(2, 2281)
(2281,)


In [215]:
X = np.vstack((np.ones(x.shape[1]),x))
idx = np.argsort(y)
y = y[idx]
X = X[:,idx]

y[y==0]= -1


In [216]:
w = np.array([0.00,0.99,0.15])

ye = np.dot(w.T,X)
Ye = ye.copy()
Ye[ye<0] = -1
Ye[ye>0] = 1

cm = confusion_matrix(y,Ye)
print(cm)
print('O número de erros na classe w0  ́e de 21: ', cm[0,1])
print('O numero de acertos na classe w1 é de 1652: ',cm[1,1])

[[ 531    5]
 [  89 1656]]
O número de erros na classe w0  ́e de 21:  5
O numero de acertos na classe w1 é de 1652:  1656


1 a) IV

In [217]:
Rx   = np.dot(X, X.T)
rxy  = np.dot(X, y.T)
wMSE = np.dot(la.pinv(Rx), rxy)

ye = np.dot(wMSE.T, X)
Ye = ye.copy()

Ye[ye<0]  = -1
Ye[ye>=0] =  1

CM = confusion_matrix(y, Ye)
print(CM)
print(' O vetor que minimiza o erro quadrático médio é wMSE = [1.254, 1.205, -0.923]', wMSE)
print('O erro quadrático médio é igual a 1.084: ', mean_squared_error(Ye,ye))

[[ 310  226]
 [   0 1745]]
 O vetor que minimiza o erro quadrático médio é wMSE = [1.254, 1.205, -0.923] [0.25374713 0.20829497 0.07654264]
O erro quadrático médio é igual a 1.084:  0.4225867831579466


1b) IV

In [218]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score


In [219]:
from sklearn import datasets
# Carregar o conjunto "diabetes"
data = datasets.load_diabetes()
X, y = data.data, data.target

# Dividir os dados em treino (316 amostras) e teste (restantes)
X_train, X_test = X[:316], X[316:]
y_train, y_test = y[:316], y[316:]

In [220]:
# Gerar características polinomiais de 2ª ordem
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Instanciar e ajustar um modelo de regressão linear
linear_model = LinearRegression()
linear_model.fit(X_train_poly, y_train)

# Previsões no conjunto de treino
y_train_pred = linear_model.predict(X_train_poly)

# Calcular métricas no conjunto de treino
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print("Regressão Polinomial de 2ª Ordem:")
print(f"No conjunto de treino, o erro absoluto médio é igual a 37.97: {mae_train:.2f}")
print(f"No conjunto de treino, o coeficiente de determinação, R**2, é igual a 0.79: {r2_train:.2f}")


Regressão Polinomial de 2ª Ordem:
No conjunto de treino, o erro absoluto médio é igual a 37.97: 37.97
No conjunto de treino, o coeficiente de determinação, R**2, é igual a 0.79: 0.61


a) I

In [221]:
# Gerar características polinomiais de 4ª ordem
poly4 = PolynomialFeatures(degree=4)
X_train_poly4 = poly4.fit_transform(X_train)
X_test_poly4 = poly4.transform(X_test)

# Instanciar e ajustar o modelo Lasso
lasso_model = Lasso(alpha=0.01, random_state=42)
lasso_model.fit(X_train_poly4, y_train)

# Previsões no conjunto de teste
y_test_pred = lasso_model.predict(X_test_poly4)

# Calcular R² no conjunto de teste
r2_test = r2_score(y_test, y_test_pred)

# Contar os coeficientes diferentes de zero (excluindo w0)
coef_count = np.sum(lasso_model.coef_ != 0)

print("\nRegressão Lasso com Polinômio de 4ª Ordem:")
print(f"No cojunto de teste, o coeficiente de determinação, R**2, é igual a 0.526: {r2_test:.3f}")
print(f"Excluindo w0, o número de coeficientes do polinómio diferentes de zero é 32: {coef_count}")



Regressão Lasso com Polinômio de 4ª Ordem:
No cojunto de teste, o coeficiente de determinação, R**2, é igual a 0.526: 0.526
Excluindo w0, o número de coeficientes do polinómio diferentes de zero é 32: 18


b) I

In [222]:
data3 = pickle.load(open('A48168_Q003_data.p','rb'))
x = data3['x']
y = data3['y']
folds = data3['folds']

In [223]:
print(x.shape)
print(y.shape)
trainX = x[folds==1]
trainY = y[folds==1]
testX = x[folds==0]
testY = y[folds==0]
print(trainX.shape)
print(testX.shape)


(318,)
(318,)
(128,)
(190,)


In [224]:
X1 = trainX[:,np.newaxis]
X2 = testX[:,np.newaxis]

poly = PolynomialFeatures(degree=3).fit(X1)
X1a  = poly.transform(X1)
X2a  = poly.transform(X2)

# Fazer a regressão linear
lr = LinearRegression().fit(X1a, trainY)
y1e = lr.predict(X1a)
y2e = lr.predict(X2a)
print('No conjunto de teste o erro quadrático médio é igual a 0.91:', round(np.mean(np.abs(testY-y2e)), 2))
print('No conjunto de treino, o coeficiente de determinação, R**2, é igual a 0.78:', round(lr.score(X1a,trainY), 2))


No conjunto de teste o erro quadrático médio é igual a 0.91: 0.81
No conjunto de treino, o coeficiente de determinação, R**2, é igual a 0.78: 0.78


a)II

In [225]:
# treino
x1 = x[folds==1]
y1 = y[folds==1]
# teste
x2 = x[folds==0]
y2 = y[folds==0]

# Adicionar 2ª dimensão
X1 = x1[:,np.newaxis]
X2 = x2[:,np.newaxis]

# Estimar polinomio com os dados de treino
poly = PolynomialFeatures(degree=4).fit(X1)
X1a  = poly.transform(X1)
X2a  = poly.transform(X2)

# Fazer a regressão linear
lr = LinearRegression().fit(X1a, y1)
y1e = lr.predict(X1a)
y2e = lr.predict(X2a)
print('No conjunto de teste, o erro absoluto médio é igual a 0.66:', round(np.mean((y1-y1e)**2), 2))
print('Arredondado a zero casas decimais, o valore de w0 é 2:', round(lr.intercept_))


No conjunto de teste, o erro absoluto médio é igual a 0.66: 0.25
Arredondado a zero casas decimais, o valore de w0 é 2: 2


b) II

In [226]:
q = pickle.load(open('A48168_Ficha4_Respostas.p','rb'))
print(q)

{'Q001': array([[0., 0., 0., 1.],
       [0., 0., 0., 1.]]), 'Q002': array([[1., 0., 0., 0.],
       [1., 0., 0., 0.]]), 'Q003': array([[0., 1., 0., 0.],
       [0., 1., 0., 0.]]), 'nome': 'Diogo Sancho Pires Lobo', 'numero': 'A48168'}


In [227]:
q['Q001'][0,:] = [0,0,0,1]
q['Q001'][1,:] = [0,0,0,1]
q['Q002'][0,:] = [1,0,0,0]
q['Q002'][1,:] = [1,0,0,0]
q['Q003'][0,:] = [0,1,0,0]
q['Q003'][1,:] = [0,1,0,0]

pickle.dump(q,open('A48168_Ficha4_Respostas.p','wb'))

In [228]:
vs = pickle.load(open('A48168_Ficha4_Respostas.p','rb'))
print(vs)

{'Q001': array([[0., 0., 0., 1.],
       [0., 0., 0., 1.]]), 'Q002': array([[1., 0., 0., 0.],
       [1., 0., 0., 0.]]), 'Q003': array([[0., 1., 0., 0.],
       [0., 1., 0., 0.]]), 'nome': 'Diogo Sancho Pires Lobo', 'numero': 'A48168'}
